Customer churn prediction is to measure why customers are leaving a business. In this tutorial we will be looking at customer churn in Bank business. We will build a deep learning model to predict the churn and use precision, recall, f1-score to measure performance of our model


In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

## LOADING THE DATASET

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## DATA CLEANING

In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## FEATURE ENGINEERING

In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
#FUNCTION TO DISPLAY CATEGORICAL VALUES
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


ONE HOT ENCODING CATEGORICAL VALUES`

In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


SCALING THE DATASET

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

# CREATING MODEL

## SPLITTING THE DATASET INTO TRAINING AND TEST SET

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [16]:
X_train.shape

(8000, 13)

In [18]:
X_test.shape

(2000, 13)

## IMPORTING TENSORFLOW LIBRARIES

In [19]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

### BUILD THE MODEL(ANN)

In [21]:
#DEFINING THE NEURAL NETWORK FUNCTION TO TRAIN THE MODEL

def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [22]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5417 - accuracy: 0.7259
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4672 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4579 - accuracy: 0.7993
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4477 - accuracy: 0.8061
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4377 - accuracy: 0.8112
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4323 - accuracy: 0.8151
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4302 - accuracy: 0.8139
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4289 - accuracy: 0.8145
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4278 - accuracy: 0.8161
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4261 - accura

As We See, the precision, recall and f1 score of Class 1 is very low due to imbalanced dataset

Mitigating Skewdness of Data

Method 1: Undersampling

reference: https: // www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets


# METHOD 1: UNDERSAMPLING FOR IMBALANCE DATASET

Undersampling is a technique used to address class imbalance in a dataset by reducing the number of instances in the majority class to match the number of instances in the minority class.

In [23]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [24]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [25]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [26]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [27]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6746 - accuracy: 0.5796
Epoch 2/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6562 - accuracy: 0.6168
Epoch 3/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6434 - accuracy: 0.6395
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6310 - accuracy: 0.6462
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6205 - accuracy: 0.6557
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6107 - accuracy: 0.6652
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6018 - accuracy: 0.6732
Epoch 8/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5922 - accuracy: 0.6876
Epoch 9/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5845 - accuracy: 0.6981
Epoch 10/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5750 - accura

As we see, there is considerable Improvement in the f1, recall and precision scores of Class 1 Value.
The f1 score has improved from 0.59 to 0.76.

## Method2: Oversampling


Oversampling is a technique used to address class imbalance in a dataset by increasing the number of instances in the minority class to match the number of instances in the majority class

In [29]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [30]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [31]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 2s 3ms/step - loss: 0.6435 - accuracy: 0.6253
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6071 - accuracy: 0.6710
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5823 - accuracy: 0.6955
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5664 - accuracy: 0.7116
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5568 - accuracy: 0.7190
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5454 - accuracy: 0.7265
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5173 - accuracy: 0.7468
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4887 - accuracy: 0.7655
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4769 - accuracy: 0.7676
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4719 - accura

f1-score for minority class 1 improved from 0.58 to 0.78.


## Method3: SMOTE

To install imbalanced-learn library use pip install imbalanced-learn command

SMOTE (Synthetic Minority Over-sampling Technique) is a popular method for addressing class imbalance by generating synthetic samples for the minority class. SMOTE creates synthetic instances by interpolating between existing minority class instances.

In [32]:
!pip install imbalanced-learn

In [33]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [34]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [36]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 3s 4ms/step - loss: 0.6520 - accuracy: 0.6210
Epoch 2/100
399/399 [==============================] - 2s 6ms/step - loss: 0.6008 - accuracy: 0.6826
Epoch 3/100
399/399 [==============================] - 2s 6ms/step - loss: 0.5815 - accuracy: 0.6980
Epoch 4/100
399/399 [==============================] - 2s 5ms/step - loss: 0.5715 - accuracy: 0.7075
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5571 - accuracy: 0.7206
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5409 - accuracy: 0.7270
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5290 - accuracy: 0.7348
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5183 - accuracy: 0.7422
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5114 - accuracy: 0.7473
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5060 - accura

SMOT Oversampling increases f1 score of minority class 1 from 0.58 to 0.80.


## Method4: Use of Ensemble with undersampling

Ensemble learning with undersampling is a technique that combines the power of ensemble methods with undersampling to address class imbalance. It involves training multiple models on different undersampled subsets of the majority class and then combining their predictions to make the final decision.

In [37]:
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [38]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [40]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [41]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [42]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [43]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [44]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 2ms/step - loss: 0.6782 - accuracy: 0.5782
Epoch 2/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6536 - accuracy: 0.6294
Epoch 3/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6375 - accuracy: 0.6429
Epoch 4/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6263 - accuracy: 0.6592
Epoch 5/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6177 - accuracy: 0.6650
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6110 - accuracy: 0.6765
Epoch 7/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6033 - accuracy: 0.6800
Epoch 8/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5966 - accuracy: 0.6826
Epoch 9/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5915 - accuracy: 0.6861
Epoch 10/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5862 - accuracy: 0.6883
Epoch 11/

In [45]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 2ms/step - loss: 0.6898 - accuracy: 0.5354
Epoch 2/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6627 - accuracy: 0.5990
Epoch 3/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6489 - accuracy: 0.6246
Epoch 4/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6385 - accuracy: 0.6410
Epoch 5/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6304 - accuracy: 0.6499
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6224 - accuracy: 0.6512
Epoch 7/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6130 - accuracy: 0.6624
Epoch 8/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6022 - accuracy: 0.6790
Epoch 9/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5961 - accuracy: 0.6845
Epoch 10/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5881 - accuracy: 0.6938
Epoch 11/

In [46]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 1s 3ms/step - loss: 0.6638 - accuracy: 0.6166
Epoch 2/100
87/87 [==============================] - 0s 2ms/step - loss: 0.6309 - accuracy: 0.6513
Epoch 3/100
87/87 [==============================] - 0s 2ms/step - loss: 0.6169 - accuracy: 0.6682
Epoch 4/100
87/87 [==============================] - 0s 2ms/step - loss: 0.6083 - accuracy: 0.6736
Epoch 5/100
87/87 [==============================] - 0s 3ms/step - loss: 0.6006 - accuracy: 0.6755
Epoch 6/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5951 - accuracy: 0.6848
Epoch 7/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5888 - accuracy: 0.6838
Epoch 8/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5837 - accuracy: 0.6877
Epoch 9/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5761 - accuracy: 0.6906
Epoch 10/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5702 - accuracy: 0.6957
Epoch 11/

In [47]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [48]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.94      0.75      0.83      1593
           1       0.45      0.81      0.58       407

    accuracy                           0.76      2000
   macro avg       0.70      0.78      0.71      2000
weighted avg       0.84      0.76      0.78      2000



f1-score for minority class 1 is 0.58